# Load Images in Grayscale

In [ ]:
# %cp -av '/content/drive/My Drive/cats' '/content/cats'
# %cp -av '/content/drive/My Drive/dogs' '/content/dogs'
!wget https://gist.githubusercontent.com/spronkoid/72007c8ed119c6cd8150b46e21ef47ca/raw/a36e70c0fd4884df1c3edb3ff2d0e9231984128f/fileloader.py

--2020-09-09 03:03:47--  https://gist.githubusercontent.com/spronkoid/72007c8ed119c6cd8150b46e21ef47ca/raw/a36e70c0fd4884df1c3edb3ff2d0e9231984128f/fileloader.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7751 (7.6K) [text/plain]
Saving to: ‘fileloader.py’

fileloader.py       100%[===================>]   7.57K  --.-KB/s    in 0.01s   

2020-09-09 03:03:47 (767 KB/s) - ‘fileloader.py’ saved [7751/7751]



In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'theano'
import numpy as np
from fileloader import *
# cats = load_raw_grey('/content/cats', 64)
# os.chdir('..')
# cats = augment(cats, horizontal_flip=True, rotations=6, vertical_flip=True, degrees=90)
# cats = unpack(cats)

# dogs = load_raw_grey('/content/dogs', 64)
# os.chdir('..')
# dogs = augment(dogs, horizontal_flip=True, rotations=6, vertical_flip=True, degrees=90)
# dogs = unpack(dogs)

# X = []
# Y = []

# for i in range(3600):
#     X.append(cats[i])
#     Y.append(1)

#     X.append(dogs[i])
#     Y.append(0)
# # X = np.array(X)
# # Y = np.array(Y)

In [ ]:
import pickle
# with open('cat_vs_dogs_train.pickle', 'wb') as f:
#     pickle.dump([X, Y], f)

with open('/content/drive/My Drive/cat_vs_dogs_train.pickle', 'rb') as f:
    X, Y = pickle.load(f)

In [ ]:
# np.array(X).shape

# Reshape images and add a placeholder for the communications

In [ ]:
x2 = X
x3 = []
for i in x2:
    x3.append(i.flatten())
# np.ones((64, 64))

In [ ]:
img_size = np.array(x3[0]).shape[0]
offset = 64

In [ ]:
x3 = np.array(x3)
placeholder_data = []
for i in x3:
    placeholder_data.append(np.append(i, np.array([0.5]*offset)))

# Create agents

In [ ]:
import keras.layers as layers
import keras.models as models

agent_one = models.Sequential()
agent_one.add(layers.Dense(65*65, input_shape=(img_size+offset,)))
agent_one.add(layers.Reshape((65,65)))
agent_one.add(layers.LocallyConnected1D(65, 3, strides=2, trainable=False))
agent_one.add(layers.LocallyConnected1D(32, 3, strides=2, trainable=False))
agent_one.add(layers.LocallyConnected1D(16, 3, strides=2))
agent_one.add(layers.Flatten())
agent_one.add(layers.Dense(1, activation='sigmoid'))

agent_one.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

agent_two = models.Sequential()
agent_two.add(layers.Dense(65*65, input_shape=(img_size+offset,)))
agent_two.add(layers.Reshape((65,65)))
agent_two.add(layers.LocallyConnected1D(65, 3, strides=2, trainable=False))
agent_two.add(layers.LocallyConnected1D(32, 3, strides=2, trainable=False))
agent_two.add(layers.LocallyConnected1D(16, 3, strides=2))
agent_two.add(layers.Flatten())
agent_two.add(layers.Dense(1, activation='sigmoid'))

agent_two.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

# Train Agents

In [ ]:
def one_cycle():
    print("="*30)
    epochs = 1

    one_data = agent_one.predict(np.array(placeholder_data))
    two_data = agent_two.predict(np.array(placeholder_data))

    placeholder_data1 = []
    for i in range(len(x3)):
        placeholder_data1.append(np.append(x3[i], np.array([one_data[i][0]]*offset)))

    placeholder_data2 = []
    for i in range(len(x3)):
        placeholder_data2.append(np.append(x3[i], np.array([two_data[i][0]]*offset)))

    agent_one.fit(np.array(placeholder_data2), np.array(Y), epochs=epochs, validation_split=0.2, verbose=0, batch_size=256)
    agent_two.fit(np.array(placeholder_data1), np.array(Y), epochs=epochs, validation_split=0.2, verbose=0, batch_size=256)

    score = agent_one.evaluate(np.array(placeholder_data2), np.array(Y), verbose = 0) 
    print('Agent One\'s loss:', score[0]) 
    print('Agent One\'s accuracy:', score[1])

    score = agent_two.evaluate(np.array(placeholder_data1), np.array(Y), verbose = 0) 
    print('Agent Two\'s loss:', score[0]) 
    print('Agent Two\'s accuracy:', score[1])

def two_cycle(agent_one, agent_two, epochs=1):
    print("="*30)
    epochs = epochs

    one_data = agent_one.predict(np.array(placeholder_data))
    two_data = agent_two.predict(np.array(placeholder_data))

    placeholder_data1 = []
    for i in range(len(x3)):
        placeholder_data1.append(np.append(x3[i], np.array([one_data[i][0]]*offset)))

    placeholder_data2 = []
    for i in range(len(x3)):
        placeholder_data2.append(np.append(x3[i], np.array([two_data[i][0]]*offset)))

    agent_one.fit(np.array(placeholder_data2), np.array(Y), epochs=epochs, validation_split=0.2, verbose=0, batch_size=256)
    agent_two.fit(np.array(placeholder_data1), np.array(Y), epochs=epochs, validation_split=0.2, verbose=0, batch_size=256)

    score = agent_one.evaluate(np.array(placeholder_data2), np.array(Y), verbose = 0) 
    print('Agent One\'s loss:', score[0], 'Agent One\'s accuracy:', score[1]*100) 
    # print('Agent One\'s accuracy:', score[1])

    score = agent_two.evaluate(np.array(placeholder_data1), np.array(Y), verbose = 0) 
    print('Agent Two\'s loss:', score[0], 'Agent Two\'s accuracy:', score[1]*100) 
    # print('Agent Two\'s accuracy:', score[1])

    ###############################################
    print("###############") # START SECOND "CYCLE"
    ###############################################

    one_data = agent_one.predict(np.array(placeholder_data2))
    two_data = agent_two.predict(np.array(placeholder_data1))

    placeholder_data1 = []
    for i in range(len(x3)):
        placeholder_data1.append(np.append(x3[i], np.array([one_data[i][0]]*offset)))

    placeholder_data2 = []
    for i in range(len(x3)):
        placeholder_data2.append(np.append(x3[i], np.array([two_data[i][0]]*offset)))

    agent_one.fit(np.array(placeholder_data2), np.array(Y), epochs=epochs, verbose=0, batch_size=256)
    agent_two.fit(np.array(placeholder_data1), np.array(Y), epochs=epochs, verbose=0, batch_size=256)

    score = agent_one.evaluate(np.array(placeholder_data2), np.array(Y), verbose = 0) 
    print('Agent One\'s loss:', score[0], 'Agent One\'s accuracy:', score[1]*100) 
    # print('Agent One\'s accuracy:', score[1])

    score = agent_two.evaluate(np.array(placeholder_data1), np.array(Y), verbose = 0) 
    print('Agent Two\'s loss:', score[0], 'Agent Two\'s accuracy:', score[1]*100) 
    # print('Agent Two\'s accuracy:', score[1])

In [ ]:
def predict(i, agent_one, agent_two):
    # return the average score between the two
    
    ao_result = agent_one.predict(np.array([np.append(i, np.array([0.5]*offset))]))
    at_result = agent_two.predict(np.array([np.append(i, np.array([0.5]*offset))]))

    ao_result2 = agent_one.predict(np.array([np.append(i, np.array([at_result[0][0]]*offset))]))
    at_result2 = agent_two.predict(np.array([np.append(i, np.array([ao_result[0][0]]*offset))]))

    ao_final = ao_result2[0][0]
    at_final = at_result2[0][0]
    # print(ao_result, ao_result2)
    return (ao_final+at_final)/2

In [ ]:
for i in range(300):
    two_cycle(agent_one, agent_two, epochs=2)

    # one_cycle()

Agent One's loss: 3.236642599105835 Agent One's accuracy: 56.6944420337677
Agent Two's loss: 4.032991409301758 Agent Two's accuracy: 50.05555748939514
###############
Agent One's loss: 0.6933689117431641 Agent One's accuracy: 58.04166793823242
Agent Two's loss: 1.859939694404602 Agent Two's accuracy: 51.80555582046509
Agent One's loss: 0.7390234470367432 Agent One's accuracy: 55.444443225860596
Agent Two's loss: 0.8232971429824829 Agent Two's accuracy: 52.04166769981384
###############
Agent One's loss: 0.657038688659668 Agent One's accuracy: 60.44444441795349
Agent Two's loss: 0.7669346928596497 Agent Two's accuracy: 53.611111640930176
Agent One's loss: 0.6356043815612793 Agent One's accuracy: 65.48610925674438
Agent Two's loss: 0.7513793110847473 Agent Two's accuracy: 55.50000071525574
###############
Agent One's loss: 0.6363468170166016 Agent One's accuracy: 65.3333306312561
Agent Two's loss: 0.6621336340904236 Agent Two's accuracy: 58.20833444595337
Agent One's loss: 0.639991402626

# Save Models

In [ ]:
agent_one.save('agent_one')
agent_two.save('agent_two')

# Load Models

In [ ]:
import keras
print("Saving Models...")
ao = keras.models.load_model("agent_one")
at = keras.models.load_model("agent_two")

In [ ]:
two_cycle(ao, at, 1)

# Use Models

In [ ]:
index = 0
to_predict = load_raw_grey('/content/test', 64, 10)[index]
os.chdir('..')
test = os.listdir('/content/test')
from google.colab.patches import cv2_imshow
import cv2
testim = cv2.imread('/content/test/'+test[index])
cv2_imshow(testim)

print("="*20, "\n", "Prediction:")
if predict(to_predict, ao, at)>0.5:
    print("Estimated Cat with " + str(predict(to_predict, ao, at)*100) + "% certainty")
else:
    print("Estimated Dog with " + str((1-predict(to_predict, ao, at))*100) + "% certainty")